# Generating RBUS subckt connections cell from connections.json file

In [268]:
import json
import os
import sys


In [269]:
inputfilenames = { "circuit_json": "../../examples/RO5_circuit.json", 
                  "pin_map": "pin_name_to_number.json"}

outputfilenames = { "output_circuit": "generated_circuit.cir"}

In [270]:
# Change the working directory to the script's directory
working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp"
os.chdir(working_dir)

# Check if the script is being run from the correct directory
for key, filename in inputfilenames.items():
    if not os.path.exists(filename):
        print(f"Error: The script must be run from the directory containing {filename}.")
        sys.exit(1)

In [271]:
def generate_circuit_file(circuit_json_path, pin_map_path, output_circuit_path):
    # Load the RO3 circuit JSON
    with open(circuit_json_path, 'r') as circuit_file:
        ro3_data = json.load(circuit_file)
    
    # Load the pin name to number mapping
    with open(pin_map_path, 'r') as pin_map_file:
        pin_name_to_number = json.load(pin_map_file)
    
    # Open the output circuit file for writing
    with open(output_circuit_path, 'w') as circuit_file:
        # Write the SPICE header
        circuit_file.write("simulator lang=spice\n\n")
        subckt_header = '''.SUBCKT PK_RBUS_external_connections RBUS<1> RBUS<2> RBUS<3> RBUS<4> RBUS<5>
+  RBUS<6> RBUS<7> RBUS<8> pin<1> pin<2> pin<3> pin<4> pin<5> pin<6> pin<7>
+  pin<8> pin<9> pin<10> pin<11> pin<12> pin<13> pin<14> pin<15> pin<16> pin<17>
+  pin<18> pin<19> pin<20> pin<21> pin<22> pin<23> pin<24> pin<25> pin<26>
+  pin<27> pin<28> pin<29> pin<30> pin<31> pin<32> pin<33> pin<34> pin<35>
+  pin<36> pin<37> pin<38> pin<39> pin<40> pin<41> pin<42> pin<43> pin<44>
+  pin<45> pin<46> pin<47> pin<48> pin<49> pin<50> pin<51> pin<52> pin<53>
+  pin<54> pin<55> pin<56> pin<57> pin<58> pin<59> pin<60> pin<61> pin<62>
+  pin<63> pin<64> pin<65> pin<66> pin<67> pin<68> pin<69> pin<70> pin<71>
+  pin<72> pin<73> pin<74> pin<75> pin<76> pin<77> pin<78> pin<79> pin<80>
+  pin<81> pin<82> pin<83> pin<84> pin<85> pin<86> pin<87> pin<88> pin<89>
+  pin<90> pin<91> pin<92> pin<93> pin<94> pin<95> pin<96> pin<97> pin<98>
+  pin<99> pin<100>
'''
        circuit_file.write(subckt_header+"\n")

        # Iterate through each RBUS in the RO3 circuit JSON
        for rbus, pin_names in ro3_data.items():
            # Extract the RBUS number (e.g., RBUS<1> -> 1)
            rbus_number = int(rbus.replace("RBUS", "").strip("<>"))
            
            # Iterate through the pin names connected to this RBUS
            for pin_name in pin_names:
                # Get the pin number from the mapping
                pin_number = pin_name_to_number.get(pin_name)
                if pin_number is not None:
                    # Write a comment with the pin name and number and the RBUS number
                    circuit_file.write(f"* {pin_name} pin<{int(pin_number)}> RBUS<{rbus_number}>\n")
                    # Write the short circuit connection to the file
                    circuit_file.write(f"Vshort_{rbus_number}_{int(pin_number)} RBUS<{rbus_number}> pin<{int(pin_number)}> 0\n")
                else:
                    print(f"Warning: Pin name '{pin_name}' not found in pin_name_to_number.json")
        
        # Write the SPICE footer
        circuit_file.write(".ENDS\n")

In [272]:
generate_circuit_file(inputfilenames["circuit_json"], inputfilenames["pin_map"], 
                      outputfilenames["output_circuit"])
